# Merging all data sources

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
import numpy as np

# add these lines to get the config to work
import json

with open('../../../../config/config.json') as config_f:
    config = json.load(config_f)


file_path=config['mimic_iv_1.0_path']
file_store_path = './preproc_output'

Load all the previously processed data from the source files in the MIMIC4 dataset. <br/>Unify column naming across data sources and merge to one dataframe.

In [ ]:
lab_df=pd.read_csv(file_store_path + "/lab_processed.csv")[["subject_id","hadm_id","charttime","valuenum","label"]]
inputs_df=pd.read_csv(file_store_path + "/inputs_processed.csv")[["subject_id","hadm_id","charttime","amount","label"]]
outputs_df=pd.read_csv(file_store_path + "/outputs_processed.csv")[["subject_id","hadm_id","charttime","value","label"]]
presc_df=pd.read_csv(file_store_path + "/prescriptions_processed.csv")[["subject_id","hadm_id","charttime","dose_val_rx","drug"]]

In [ ]:
#Change the name of amount. Valuenum for every table
inputs_df["valuenum"]=inputs_df["amount"]
inputs_df.head()
inputs_df=inputs_df.drop(columns=["amount"]).copy()

outputs_df["valuenum"]=outputs_df["value"]
outputs_df=outputs_df.drop(columns=["value"]).copy()

presc_df["valuenum"]=presc_df["dose_val_rx"]
presc_df=presc_df.drop(columns=["dose_val_rx"]).copy()
presc_df["label"]=presc_df["drug"]
presc_df=presc_df.drop(columns=["drug"]).copy()
#presc_df = presc_df.drop((presc_df['valuenum']=='3-10').index)
presc_df = presc_df[~presc_df['valuenum'].isin(['3-10', '10-20', '5-10', '2-4', '20-40', '40-60', '2.5-5', '20-60',
       '100-200', '5-20', '500-1000', '1-10', '2.5 - 5', '10-15', '15-20',
       '250-500', '100-250', '5-15', '2.5-7.5', '2.5-10', '1.25-2.5',
       '1-20'])]

#Tag to distinguish between lab and inputs events
inputs_df["Origin"]="Inputs"
lab_df["Origin"]="Lab"
outputs_df["Origin"]="Outputs"
presc_df["Origin"]="Prescriptions"

#merge both dfs.
merged_df1=(inputs_df.append(lab_df)).reset_index()
merged_df2=(merged_df1.append(outputs_df)).reset_index()
merged_df2.drop(columns="level_0",inplace=True)
merged_df=(merged_df2.append(presc_df)).reset_index()

#Check that all labels have different names.
assert(merged_df["label"].nunique()==(inputs_df["label"].nunique()+lab_df["label"].nunique()+outputs_df["label"].nunique()+presc_df["label"].nunique()))

In [4]:
# set the timestamp as the time delta between the first chart time for each admission
merged_df['charttime']=pd.to_datetime(merged_df["charttime"], format='%Y-%m-%d %H:%M:%S')
ref_time=merged_df.groupby("hadm_id")["charttime"].min()
merged_df_1=pd.merge(ref_time.to_frame(name="ref_time"),merged_df,left_index=True,right_on="hadm_id")
merged_df_1["time_stamp"]=merged_df_1["charttime"]-merged_df_1["ref_time"]
assert(len(merged_df_1.loc[merged_df_1["time_stamp"]<timedelta(hours=0)].index)==0)

In [5]:
# Create a label code (int) for the labels.
label_dict=dict(zip(list(merged_df_1["label"].unique()),range(len(list(merged_df_1["label"].unique())))))
merged_df_1["label_code"]=merged_df_1["label"].map(label_dict)

merged_df_short=merged_df_1[["hadm_id","valuenum","time_stamp","label_code","Origin"]]

In [6]:
label_dict_df=pd.Series(merged_df_1["label"].unique()).reset_index()
label_dict_df.columns=["index","label"]
label_dict_df["label_code"]=label_dict_df["label"].map(label_dict)
label_dict_df.drop(columns=["index"],inplace=True)
label_dict_df.to_csv(file_store_path + "/label_dict.csv")

In [ ]:
merged_df_short["valuenum"] = merged_df_short["valuenum"].astype(float)

In [ ]:
# select only values within first 48 hours
merged_df_short=merged_df_short.loc[(merged_df_short["time_stamp"]<timedelta(hours=48))]
merged_df_short["time_stamp"] = merged_df_short["time_stamp"].dt.total_seconds().div(60).astype(int)
print("Number of patients considered: "+str(merged_df_short["hadm_id"].nunique()))
assert(len(merged_df_short.loc[merged_df_short["time_stamp"]>2880].index)==0)

In [ ]:
# drop columns that are not needed for final dataset
merged_df_short.drop(["Origin"], axis=1, inplace=True)
complete_df = merged_df_short

In [ ]:
# create value- and mask- columns and fill with data
labels = complete_df["label_code"].unique()
value_columns = []
mask_columns  = []
for num in labels:
    name = "Value_label_" + str(num)
    name2 = "Mask_label_" + str(num)
    value_columns.append(name)
    mask_columns.append(name2)
    complete_df[name] = 0
    complete_df[name2] = 0
    complete_df[name] = complete_df[name].astype(float)

In [ ]:
complete_df.dropna(inplace=True)
for index, row in complete_df.iterrows():
    name = "Value_label_" + str(row["label_code"].astype(int))
    name2 = "Mask_label_" + str(row["label_code"].astype(int))
    complete_df.at[index, name] = row["valuenum"]
    complete_df.at[index, name2] = 1

In [ ]:
# drop all unneccesary columns and do sanity check
complete_df.drop(["valuenum", "label_code"], axis=1, inplace=True)
complete_df = complete_df.groupby(["hadm_id", "time_stamp"], as_index=False).max()
for x in mask_columns:
    assert(len(complete_df.loc[complete_df[x]>1])==0)
complete_df

To prepare for tPatchGNN processing:

In [ ]:
#complete_df.to_csv(file_store_path + "/full_dataset.csv", index=False)

In [19]:
complete_df.rename(columns={"hadm_id": "ID", "time_stamp": "Time"}, inplace=True)

In [21]:
complete_df.to_csv(file_store_path + "/full_dataset.csv", index=False)